In [18]:
# 1. Getting Started - Load Python Packages
# 1.1. Import Packages
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use("Agg")
import sys

os.chdir("../../../DRLTrading")

import warnings
warnings.filterwarnings('ignore')
import pyfolio as pf

from meta import config
from meta import config_tickers
from meta.data_processor import DataProcessor
sys.path.append("C:\python310\lib\site-packages")
from meta.env_portfolio_allocation.env_portfolio_yahoofinance import (
    StockPortfolioEnv,
)
from agents.stablebaselines3_models import DRLAgent
from plot import (
    backtest_stats,
    backtest_plot,
    get_daily_return,
    get_baseline,
    convert_daily_return_to_pyfolio_ts,
)

In [19]:
# 1.2. Create Folders
import main

main.check_and_make_directories(
    [
        config.DATA_SAVE_DIR,
        config.TRAINED_MODEL_DIR,
        config.TENSORBOARD_LOG_DIR,
        config.RESULTS_DIR,
    ]
)

In [9]:
# 2. Download and Preprocess Data
print(f"DOW_30_TICKER: {config_tickers.DOW_30_TICKER}")

dp = DataProcessor(
    data_source="yahoofinance",
    start_date="2009-01-01",
    end_date="2019-01-01",
    time_interval="1D",
)

dp.run(
    ticker_list=config_tickers.DOW_30_TICKER,
    technical_indicator_list=config.INDICATORS,
    if_vix=False,
)
df = dp.dataframe

df.head()

print("Shape of DataFrame: ", df.shape)

# Add covariance matrix as states
df.rename(columns={"time": "date"}, inplace=True)
df = df.sort_values(["date", "tic"], ignore_index=True)
df.index = df.date.factorize()[0]
df.drop(columns=["index"], inplace=True)

cov_list = []
return_list = []

# look back is one year
lookback = 252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i - lookback : i, :]
    price_lookback = data_lookback.pivot_table(
        index="date", columns="tic", values="close"
    ).dropna(axis=1)
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)

df_cov = pd.DataFrame(
    {
        "date": df.date.unique()[lookback:],
        "cov_list": cov_list,
        "return_list": return_list,
    }
)
df = df.merge(df_cov, on="date")
df = df.sort_values(["date", "tic"]).reset_index(drop=True)
print("Shape of DataFrame: ", df.shape)

df.head()

DOW_30_TICKER: ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']
yahoofinance successfully connected
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************] 

,date,open,high,low,close,adjusted_close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.422665,552160000.0,AAPL,0.151770,7.809801,6.634520,59.459729,119.669328,25.355894,7.180202,7.115042,"[[0.00043702812754458194, 0.000134165767139395...",tic AAPL AMGN AXP ...
1,2010-01-06,56.939999,57.389999,56.500000,56.790001,42.570049,6015100.0,AMGN,0.271541,58.391582,54.840419,50.007936,27.581378,1.908627,56.658667,56.394500,"[[0.00043702812754458194, 0.000134165767139395...",tic AAPL AMGN AXP ...
2,2010-01-06,41.230000,41.669998,41.169998,41.490002,34.263405,8399400.0,AXP,0.354226,41.997035,39.628965,58.194409,79.143497,20.617391,40.788000,39.015000,"[[0.00043702812754458194, 0.000134165767139395...",tic AAPL AMGN AXP ...
3,2010-01-06,58.230000,59.990002,57.880001,59.779999,46.582798,8836500.0,BA,1.101808,58.219951,52.756048,65.109172,257.267429,40.596881,54.797333,52.651667,"[[0.00043702812754458194, 0.000134165767139395...",tic AAPL AMGN AXP ...
4,2010-01-06,59.180000,59.930000,59.049999,59.430000,41.974724,4718800.0,CAT,0.283309,59.574505,56.181494,56.931953,152.077629,16.584805,58.042000,57.683000,"[[0.00043702812754458194, 0.000134165767139395...",tic AAPL AMGN AXP ...


In [10]:
# 4. Design Environment

# Training data split: 2009-01-01 to 2018-01-01
train = dp.data_split(df, "2009-01-01", "2018-01-01")

train.head()

# Environment for Portfolio Allocation
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
}

e_train_gym = StockPortfolioEnv(df=train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()
# print(type(env_train))

Stock Dimension: 29, State Space: 29


In [14]:
# 5. Implement DRL Algorithms

# initialize
agent = DRLAgent(env=env_train)

# Model 1: A2C
agent = DRLAgent(env=env_train)
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c", model_kwargs=A2C_PARAMS)
trained_a2c = agent.train_model(
    model=model_a2c, tb_log_name="a2c", total_timesteps=50000
)
trained_a2c.save("/DRLTrading/trained_models/trained_a2c.zip")

# Model 2: PPO
agent = DRLAgent(env=env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo", model_kwargs=PPO_PARAMS)
trained_ppo = agent.train_model(
    model=model_ppo, tb_log_name="ppo", total_timesteps=80000
)
trained_ppo.save("/DRLTrading/trained_models/trained_ppo.zip")

# Model 3: DDPG
agent = DRLAgent(env=env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS)
trained_ddpg = agent.train_model(
    model=model_ddpg, tb_log_name="ddpg", total_timesteps=50000
)
trained_ddpg.save("/DRLTrading/trained_models/trained_ddpg.zip")

# Model 4: SAC
agent = DRLAgent(env=env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}
model_sac = agent.get_model("sac", model_kwargs=SAC_PARAMS)
trained_sac = agent.train_model(
    model=model_sac, tb_log_name="sac", total_timesteps=50000
)
trained_sac.save("/DRLTrading/trained_models/trained_sac.zip")

# Model 5: TD3
agent = DRLAgent(env=env_train)
TD3_PARAMS = {"batch_size": 100, "buffer_size": 1000000, "learning_rate": 0.001}
model_td3 = agent.get_model("td3", model_kwargs=TD3_PARAMS)
trained_td3 = agent.train_model(
    model=model_td3, tb_log_name="td3", total_timesteps=30000
)
trained_td3.save("/DRLTrading/trained_models/trained_td3.zip")

# Trading
trade = dp.data_split(df, "2018-01-01", "2019-01-01")
e_trade_gym = StockPortfolioEnv(df=trade, **env_kwargs)

print("Shape of Trade DataFrame: ", trade.shape)

df_daily_return, df_actions = DRLAgent.DRL_prediction(
    model=trained_a2c, environment=e_trade_gym
)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device
Logging to tensorboard_log/a2c\a2c_2
-------------------------------------
| time/                 |           |
|    fps                | 193       |
|    iterations         | 100       |
|    time_elapsed       | 2         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -41.1     |
|    explained_variance | -2.38e-07 |
|    learning_rate      | 0.0002    |
|    n_updates          | 99        |
|    policy_loss        | 1.4e+08   |
|    reward             | 1211942.2 |
|    std                | 0.998     |
|    value_loss         | 1.49e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 178       |
|    iterations         | 200       |
|    time_elapsed       | 5         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_

-------------------------------------
| time/                 |           |
|    fps                | 249       |
|    iterations         | 1400      |
|    time_elapsed       | 28        |
|    total_timesteps    | 7000      |
| train/                |           |
|    entropy_loss       | -40.9     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0002    |
|    n_updates          | 1399      |
|    policy_loss        | 2.14e+08  |
|    reward             | 1853078.9 |
|    std                | 0.992     |
|    value_loss         | 3.68e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 250       |
|    iterations         | 1500      |
|    time_elapsed       | 29        |
|    total_timesteps    | 7500      |
| train/                |           |
|    entropy_loss       | -40.9     |
|    explained_variance | 0         |
|    learning_rate      | 0.0002    |
|    n_updat

-------------------------------------
| time/                 |           |
|    fps                | 260       |
|    iterations         | 2700      |
|    time_elapsed       | 51        |
|    total_timesteps    | 13500     |
| train/                |           |
|    entropy_loss       | -40.8     |
|    explained_variance | 0         |
|    learning_rate      | 0.0002    |
|    n_updates          | 2699      |
|    policy_loss        | 2.13e+08  |
|    reward             | 1973230.6 |
|    std                | 0.986     |
|    value_loss         | 3.99e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 261       |
|    iterations         | 2800      |
|    time_elapsed       | 53        |
|    total_timesteps    | 14000     |
| train/                |           |
|    entropy_loss       | -40.7     |
|    explained_variance | 0         |
|    learning_rate      | 0.0002    |
|    n_updat

-------------------------------------
| time/                 |           |
|    fps                | 267       |
|    iterations         | 4000      |
|    time_elapsed       | 74        |
|    total_timesteps    | 20000     |
| train/                |           |
|    entropy_loss       | -40.6     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0002    |
|    n_updates          | 3999      |
|    policy_loss        | 2.99e+08  |
|    reward             | 2480461.8 |
|    std                | 0.982     |
|    value_loss         | 6.46e+13  |
-------------------------------------
begin_total_asset:1000000
end_total_asset:2810562.807004213
Sharpe:  0.9936448349643041
-------------------------------------
| time/                 |           |
|    fps                | 266       |
|    iterations         | 4100      |
|    time_elapsed       | 76        |
|    total_timesteps    | 20500     |
| train/                |           |
|    entropy_loss       | -40.6     |


begin_total_asset:1000000
end_total_asset:3330618.2938814075
Sharpe:  1.1472301370767604
-------------------------------------
| time/                 |           |
|    fps                | 268       |
|    iterations         | 5300      |
|    time_elapsed       | 98        |
|    total_timesteps    | 26500     |
| train/                |           |
|    entropy_loss       | -40.5     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0002    |
|    n_updates          | 5299      |
|    policy_loss        | 1.27e+08  |
|    reward             | 1146367.0 |
|    std                | 0.977     |
|    value_loss         | 1.48e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 269       |
|    iterations         | 5400      |
|    time_elapsed       | 100       |
|    total_timesteps    | 27000     |
| train/                |           |
|    entropy_loss       | -40.4     |

...

begin_total_asset:1000000
end_total_asset:3270265.614047258
Sharpe:  1.1286611072021393
---------------------------------------
| time/                   |           |
|    fps                  | 291       |
|    iterations           | 31        |
|    time_elapsed         | 218       |
|    total_timesteps      | 63488     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -41.1     |
|    explained_variance   | -1.19e-07 |
|    learning_rate        | 0.0001    |
|    loss                 | 4.5e+14   |
|    n_updates            | 300       |
|    policy_gradient_loss | -9.61e-07 |
|    reward               | 2011958.9 |
|    std                  | 1         |
|    value_loss           | 9.54e+14  |
---------------------------------------
begin_total_asset:1000000
end_total_asset:3188552.436611934
Sharpe:  1.1080246114223635
------------------------

begin_total_asset:1000000
end_total_asset:3192913.9611749914
Sharpe:  1.1051292522851703
---------------------------------------
| time/                   |           |
|    fps                  | 287       |
|    iterations           | 40        |
|    time_elapsed         | 285       |
|    total_timesteps      | 81920     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -41.1     |
|    explained_variance   | 2.38e-07  |
|    learning_rate        | 0.0001    |
|    loss                 | 4.9e+14   |
|    n_updates            | 390       |
|    policy_gradient_loss | -7.54e-07 |
|    reward               | 2139684.5 |
|    std                  | 1         |
|    value_loss           | 9.46e+14  |
---------------------------------------
{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device
Logging to tensorboard_log/d

begin_total_asset:1000000
end_total_asset:3009595.111485344
Sharpe:  1.054803211965448
begin_total_asset:1000000
end_total_asset:3009492.8403920573
Sharpe:  1.0547738043034598
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 20        |
|    time_elapsed    | 389       |
|    total_timesteps | 8044      |
| train/             |           |
|    actor_loss      | -5.36e+07 |
|    critic_loss     | 2.45e+11  |
|    ent_coef        | 1.18      |
|    ent_coef_loss   | -30.2     |
|    learning_rate   | 0.0003    |
|    n_updates       | 7943      |
|    reward          | 3009492.8 |
----------------------------------
begin_total_asset:1000000
end_total_asset:3009534.051783207
Sharpe:  1.054784219185059
begin_total_asset:1000000
end_total_asset:3009581.431688734
Sharpe:  1.0548029901734852
begin_total_asset:1000000
end_total_asset:3009543.6667398415
Sharpe:  1.0547914635446556
begin_total_asset:1000000
end_total

begin_total_asset:1000000
end_total_asset:3010096.110252478
Sharpe:  1.0423906295420917
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 25        |
|    time_elapsed    | 621       |
|    total_timesteps | 16088     |
| train/             |           |
|    actor_loss      | -4.83e+07 |
|    critic_loss     | 4.14e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 14077     |
|    reward          | 3010096.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:3010096.110252478
Sharpe:  1.0423906295420917
begin_total_asset:1000000
end_total_asset:3010096.110252478
Sharpe:  1.0423906295420917
begin_total_asset:1000000
end_total_asset:3010096.110252478
Sharpe:  1.0423906295420917
begin_total_asset:1000000
end_total_asset:3010096.110252478
Sharpe:  1.0423906295420917
----------------------------------
| time/              |           |
|    episodes        | 12        |


FileNotFoundError: [Errno 2] No such file or directory: '/DRLTrading/results/df_daily_return.csv'

In [25]:
print(df_daily_return.head())

df_daily_return.to_csv("results/df_daily_return.csv")

print(df_actions.head())

df_actions.to_csv("results/df_actions.csv")

         date  daily_return
0  2018-01-02      0.000000
1  2018-01-03      0.001633
2  2018-01-04      0.003204
3  2018-01-05      0.008419
4  2018-01-08      0.000566
                AAPL      AMGN       AXP        BA       CAT       CRM  \
date                                                                     
2018-01-02  0.034483  0.034483  0.034483  0.034483  0.034483  0.034483   
2018-01-03  0.062133  0.024628  0.024628  0.046709  0.030228  0.042598   
2018-01-04  0.022867  0.022867  0.028693  0.040923  0.022969  0.045719   
2018-01-05  0.026071  0.068291  0.037813  0.070867  0.047386  0.026071   
2018-01-08  0.019938  0.047013  0.054197  0.019938  0.054197  0.052469   

                CSCO       CVX       DIS        GS  ...       MRK      MSFT  \
date                                                ...                       
2018-01-02  0.034483  0.034483  0.034483  0.034483  ...  0.034483  0.034483   
2018-01-03  0.036491  0.024628  0.024628  0.024628  ...  0.024628  0.024628 

In [28]:
# 6. Backtest Our Strategy

# 6.1. BackTestStats
from pyfolio import timeseries

DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats
perf_stats_all = perf_func(
    returns=DRL_strat,
    factor_returns=DRL_strat,
    positions=None,
    transactions=None,
    turnover_denom="AGB",
)

print("==============DRL Strategy Stats===========")
print(perf_stats_all)

# baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
    ticker="^DJI",
    start=df_daily_return.loc[0, "date"],
    end=df_daily_return.loc[len(df_daily_return) - 1, "date"],
)

stats = backtest_stats(baseline_df, value_col_name="close")

==============DRL Strategy Stats===========
Annual return          0.013342
Cumulative returns     0.013289
Annual volatility      0.172306
Sharpe ratio           0.162973
Calmar ratio           0.089409
Stability              0.317127
Max drawdown          -0.149226
Omega ratio            1.029815
Sortino ratio          0.220579
Skew                  -0.355624
Kurtosis               3.369572
Tail ratio             0.736978
Daily value at risk   -0.021597
Alpha                  0.000000
Beta                   1.000000
dtype: float64
==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (250, 8)
Annual return         -0.071511
Cumulative returns    -0.070964
Annual volatility      0.179326
Sharpe ratio          -0.325705
Calmar ratio          -0.380947
Stability              0.001178
Max drawdown          -0.187719
Omega ratio            0.943841
Sortino ratio         -0.427754
Skew                        Na

In [35]:
# 6.2. BackTestPlot
import pyfolio

baseline_df = get_baseline(
    ticker="^DJI", start=df_daily_return.loc[0, "date"], end="2021-11-01"
)

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

# with pyfolio.plotting.plotting_context(font_scale=1.1):
#     pyfolio.create_full_tear_sheet(
#         returns=DRL_strat, benchmark_rets=baseline_returns, set_context=False
#     )

# Min-Variance Portfolio Allocation
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

df.head()

# Calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
initial_capital = 1000000
portfolio.loc[0, unique_trade_date[0]] = initial_capital

for i in range(len(unique_trade_date) - 1):
    df_temp = df[df.date == unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date == unique_trade_date[i + 1]].reset_index(drop=True)
    # Sigma = risk_models.sample_cov(df_temp.return_list[0])

    # calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()

    # portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma, weight_bounds=(0, 0.1))

    # minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()

    # get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()

    # current capital
    cap = portfolio.iloc[0, i]

    # current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]

    # current held shares
    current_shares = list(np.array(current_cash) / np.array(df_temp.close))

    # next time period price
    next_price = np.array(df_temp_next.close)

    ##next_price * current share to calculate next total account value
    portfolio.iloc[0, i + 1] = np.dot(current_shares, next_price)

portfolio = portfolio.T
portfolio.columns = ["account_value"]

portfolio.head()

a2c_cumpod = (df_daily_return.daily_return + 1).cumprod() - 1

min_var_cumpod = (portfolio.account_value.pct_change() + 1).cumprod() - 1

dji_cumpod = (baseline_returns + 1).cumprod() - 1

# Plotly: DRL, Min-Variance, DJIA
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

time_ind = pd.Series(df_daily_return.date)

trace0_portfolio = go.Scatter(
    x=time_ind, y=a2c_cumpod, mode="lines", name="A2C (Portfolio Allocation)"
)

trace1_portfolio = go.Scatter(x=time_ind, y=dji_cumpod, mode="lines", name="DJIA")
trace2_portfolio = go.Scatter(
    x=time_ind, y=min_var_cumpod, mode="lines", name="Min-Variance"
)
# trace3_portfolio = go.Scatter(x = time_ind, y = ddpg_cumpod, mode = 'lines', name = 'DDPG')
# trace4_portfolio = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')
# trace5_portfolio = go.Scatter(x = time_ind, y = min_cumpod, mode = 'lines', name = 'Min-Variance')
# trace4 = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')
# trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
# trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

fig = go.Figure()
fig.add_trace(trace0_portfolio)
fig.add_trace(trace1_portfolio)
fig.add_trace(trace2_portfolio)

fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(family="sans-serif", size=15, color="black"),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2,
    ),
)

# fig.update_layout(legend_orientation="h")

fig.update_layout(
    title={
        #'text': "Cumulative Return using FinRL",
        "y": 0.85,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    }
)

# with Transaction cost
# fig.update_layout(title =  'Quarterly Trade Date')

fig.update_layout(
    #    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="rgba(1,1,0,0)",
    plot_bgcolor="rgba(1, 1, 0, 0)",
    # xaxis_title="Date",
    yaxis_title="Cumulative Return",
    xaxis={
        "type": "date",
        "tick0": time_ind[0],
        "tickmode": "linear",
        "dtick": 86400000.0 * 80,
    },
)
fig.update_xaxes(
    showline=True,
    linecolor="black",
    showgrid=True,
    gridwidth=1,
    gridcolor="LightSteelBlue",
    mirror=True,
)
fig.update_yaxes(
    showline=True,
    linecolor="black",
    showgrid=True,
    gridwidth=1,
    gridcolor="LightSteelBlue",
    mirror=True,
)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor="LightSteelBlue")

fig.show()

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (965, 8)


Start date,2018-01-02
End date,2018-12-31
Total months,11
,Backtest
Annual return,1.3%
Cumulative returns,1.3%
Annual volatility,17.2%
Sharpe ratio,0.16
Calmar ratio,0.09
Stability,0.32
Max drawdown,-14.9%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'